# Para creacion en AWS

## Dependencias

In [ ]:
pip install boto3

In [ ]:
pip install sagemaker

## Carga de los datos

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('./cars/all_aws.csv')

train_data, test_data, _ = np.split(data.sample(frac=1, random_state=123),[int(0.70*len(data)), int(len(data))])

train_data.to_csv('train_all_aws.csv', index=False, header=True, sep=',')
test_data.to_csv('test_all_aws.csv', index=False, header=True, sep=',')

data.tail(5)

## Subida de los datos a S3

In [ ]:
from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

In [ ]:
import sagemaker
session = sagemaker.Session()
uri = session.upload_data(path='train_all_aws.csv', key_prefix='sagemaker/cochesnet/input')
print(uri)

## Prueba del cliente desplegado

In [ ]:
#una vez que se haya obtenido el modelo, se despliega, y se prueba
import boto3
import sys

client = boto3.Session().client('runtime.sagemaker', region_name='us-east-1')
endpoint = 'CochesNetAwsExpEndPoint'

data_test = pd.read_csv('test_all_aws.csv')

data_x_test = data_test.drop('price', axis = 1)
data_y_test = data_test['price']
column_names=",".join(list(data_x_test.columns))

for index, row in data_x_test.iterrows():

    line=column_names+"\n"+",".join([str(e) for e in row.tolist()])

    response = client.invoke_endpoint(EndpointName=endpoint, ContentType='text/csv', Accept='text/csv', Body=line)
    response = response['Body'].read().decode('utf-8')
    response = response.split(',')[1]

    right=data_test.iloc[index]['price'].tolist()

    print(f'Response: {response} || Right: {right}')
    print(f'Response: {response} || Right: {right}')
    break